In [1]:
import math
import os
import shutil
import time
import numpy as np
from datetime import datetime
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.models import vgg16
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from HSI_class import HSI
import createSample as CS
import augmentation as aug

import simsiam.loader
import simsiam.builder
import random

start_time = time.time()

# Check if GPU is available
print("GPU Available:", torch.cuda.is_available())

# If available, print the GPU name
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
    
sample_per_class = 5
num_per_category_augment_1 = 10
num_per_category_augment_2 = 10
epochs = 200

batch_size = 20
test_size = 0.5

random_indice = 1

seeded_run = False


GPU Available: True
GPU Name: NVIDIA GeForce RTX 4080 SUPER


In [2]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    # PyTorch determinism
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    os.environ['PYTHONHASHSEED'] = str(seed)

if seeded_run:
    set_seed(42)
    print("seed has been set")

In [3]:
model_names = sorted(name for name in models.__dict__
    if name.islower() and not name.startswith("__")
    and callable(models.__dict__[name]))

print(model_names)
# create model
arch = 'vgg16' 
print("=> creating model '{}'".format(arch))
model = simsiam.builder.SimSiam(
    models.__dict__[arch])


lr = 0.01
init_lr = lr * batch_size / 256
gpu = 0

print(model)

['alexnet', 'convnext_base', 'convnext_large', 'convnext_small', 'convnext_tiny', 'densenet121', 'densenet161', 'densenet169', 'densenet201', 'efficientnet_b0', 'efficientnet_b1', 'efficientnet_b2', 'efficientnet_b3', 'efficientnet_b4', 'efficientnet_b5', 'efficientnet_b6', 'efficientnet_b7', 'efficientnet_v2_l', 'efficientnet_v2_m', 'efficientnet_v2_s', 'get_model', 'get_model_builder', 'get_model_weights', 'get_weight', 'googlenet', 'inception_v3', 'list_models', 'maxvit_t', 'mnasnet0_5', 'mnasnet0_75', 'mnasnet1_0', 'mnasnet1_3', 'mobilenet_v2', 'mobilenet_v3_large', 'mobilenet_v3_small', 'regnet_x_16gf', 'regnet_x_1_6gf', 'regnet_x_32gf', 'regnet_x_3_2gf', 'regnet_x_400mf', 'regnet_x_800mf', 'regnet_x_8gf', 'regnet_y_128gf', 'regnet_y_16gf', 'regnet_y_1_6gf', 'regnet_y_32gf', 'regnet_y_3_2gf', 'regnet_y_400mf', 'regnet_y_800mf', 'regnet_y_8gf', 'resnet101', 'resnet152', 'resnet18', 'resnet34', 'resnet50', 'resnext101_32x8d', 'resnext101_64x4d', 'resnext50_32x4d', 'shufflenet_v2_x0_

d:\FathanAbi\fathanvenv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\FathanAbi\fathanvenv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


SimSiam(
  (pre_conv): Sequential(
    (0): Conv2d(224, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): AdaptiveAvgPool2d(output_size=(1, 1))
  )
  (fc): Linear(in_features=256, out_features=200704, bias=True)
  (encoder): VGG(
    (features): Sequential(
      (0): ReLU(inplace=True)
      (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): ReLU(inplace=True)
      (6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True

In [4]:
dataset_path = r"D:\FathanAbi\tugas-akhir-model-deteksi-tumpahan-minyakl\Hyperspectral oil spill detection datasets"

dataset = []

i = 0
for filename in os.listdir(dataset_path):
    if i > 0:
        break
    file_path = os.path.join(dataset_path, filename)
    if os.path.isfile(file_path):  # Check if it's a file
        print(f"Processing file: {file_path}")
        hsi = HSI(file_path)
        dataset.append(hsi)
    i += 1

hsi_ = dataset[0]
patch_size = 9
sample_per_class = sample_per_class

indices_0 = []
indices_1 = []

print(f"random: {random_indice}")

if random_indice:
    print("generating random sample")
    selected_patch_0, selected_patch_1, indices_0, indices_1 = CS.createSample(hsi_, patch_size, sample_per_class)
else:
    print("using generated indices")
    indices_0 = [(np.int64(188), np.int64(124)), (np.int64(523), np.int64(150)), (np.int64(1003), np.int64(474)), (np.int64(616), np.int64(508)), (np.int64(905), np.int64(552))]
    indices_1 = [(np.int64(106), np.int64(606)), (np.int64(297), np.int64(468)), (np.int64(926), np.int64(35)), (np.int64(536), np.int64(519)), (np.int64(508), np.int64(442))]

    selected_patch_0, selected_patch_1 = CS.getSample(hsi_, patch_size, sample_per_class, indices_0, indices_1)


i =0
half_patch = patch_size // 2

indices = indices_0 +  indices_1

# Concatenating along axis 0
x_train = np.concatenate((selected_patch_0, selected_patch_1), )

y_train = np.array([])

gt = hsi_.gt
for indice in indices:
    # print(gt[indice[0]][indice[1]])
    y_train = np.append(y_train, gt[indice[0]][indice[1]])

count = np.count_nonzero(y_train == 0)  # Count elements equal to 0
print(f'number of element equal 0 {count}')

count = np.count_nonzero(y_train == 1)  # Count elements equal to 1
print(f'number of element equal 1 {count}')



# Print shape to verify
print(f"x_train shape: {x_train.shape}")  # Expected output: (10, 9, 9, 224)
print(f"y_train shape: {y_train.shape}") 




Processing file: D:\FathanAbi\tugas-akhir-model-deteksi-tumpahan-minyakl\Hyperspectral oil spill detection datasets\GM01.mat
random: 1
generating random sample
hsi shape
(1243, 684, 224)
creating 5 Randomly chosen 0 indices:
creating 5 Randomly chosen 1 indices:
indices 0 used: [(np.int64(509), np.int64(422)), (np.int64(111), np.int64(334)), (np.int64(795), np.int64(130)), (np.int64(324), np.int64(229)), (np.int64(112), np.int64(357))]
indices 1 used: [(np.int64(983), np.int64(76)), (np.int64(136), np.int64(51)), (np.int64(1123), np.int64(206)), (np.int64(17), np.int64(439)), (np.int64(398), np.int64(554))]
number of element equal 0 5
number of element equal 1 5
x_train shape: (10, 9, 9, 224)
y_train shape: (10,)


In [5]:
i =1
half_patch = patch_size // 2
print(hsi_.img[indices_0[i][0]][indices_0[i][1]])
print(selected_patch_0[i][half_patch][half_patch])

print(hsi_.img[indices_1[i][0]][indices_1[i][1]])
print(selected_patch_1[i][half_patch][half_patch])
i =4
half_patch = patch_size // 2
print(hsi_.img[indices_0[i][0]][indices_0[i][1]])
print(selected_patch_0[i][half_patch][half_patch])

print(hsi_.img[indices_1[i][0]][indices_1[i][1]])
print(selected_patch_1[i][half_patch][half_patch])

[-226 -170  410  415  451  576  652  664  626  583  543  496  470  429
  413  378  355  332  313  285  256  240  217  194  170  158  154  144
  136  128  130  122  123  116  104   88   75   64   54   16   36   49
   54   20   36   41   36   29   26   11   19   19   36   30   29   27
   23   13    1    0    1  -13 -127  -43   22   17   27   30   33   35
   37   33   37   31   30   29   26   23   16   -7 -174 -222  -89 -109
  -20    7   18   12   27   17   30   31   37   37   40   47   42   48
   36   46   24   18   18  -60    6  -10    0    0    0    0    0    0
    0    0 -151 -132  -52  -59  -30  -32   11   12   17   20   24    1
   27   28   28   33   29   26   32   32   33   26   27   21   29   34
   32   40   42   22   29   13   18   13    7   -1   -8  -70    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
  -66  -28    0    6   10    7    2   11   21   23   21   24   27   32
   33   32   34   29   32   31   39   32   35   35   33   31   29   27
   36 

In [6]:
n_category = 2
band_size = 224
num_per_category_augment_1 = num_per_category_augment_1
num_per_category_augment_2 = num_per_category_augment_2

data_augment1, label_augment1 = aug.Augment_data(x_train, y_train, n_category, patch_size, band_size, num_per_category_augment_1)

data_augment2, label_augment2 = aug.Augment_data2(x_train, y_train, n_category, patch_size, band_size, num_per_category_augment_2)

print(f"hasil augmentasi 1 shape: {data_augment1.shape}")
print(f"label augmentai 1 shape: {label_augment1.shape}")

print(f"hasil augmentasi 2 shape: {data_augment2.shape}")
print(f"label augmentasi 2 shape: {label_augment2.shape}")

print(label_augment1)
print(label_augment2)

# # Count occurrences of each unique element
# counts1 = np.bincount(label_augment1)

# # Print results
# for i, count in enumerate(counts1):
#     print(f"Element {i} occurs {count} times.")

# counts2 = np.bincount(label_augment2)

# # Print results
# for i, count in enumerate(counts2):
#     print(f"Element {i} occurs {count} times.")

# print(label_augment1[3])

data_augment = np.concatenate((data_augment1, data_augment2))
label_augment = np.concatenate((label_augment1, label_augment2))

print(f"hasil augmentasi gabungan untuk training: {data_augment.shape}")
print(f"label augmentasi gabungan: {label_augment.shape}")

# print(label_augment)

# Count occurrences of each unique element
counts = np.bincount(label_augment)

# Print results
for i, count in enumerate(counts):
    print(f"Element {i} occurs {count} times.")

hasil augmentasi 1 shape: (20, 9, 9, 224)
label augmentai 1 shape: (20,)
hasil augmentasi 2 shape: (20, 9, 9, 224)
label augmentasi 2 shape: (20,)
[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1]
[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1]
hasil augmentasi gabungan untuk training: (40, 9, 9, 224)
label augmentasi gabungan: (40,)
Element 0 occurs 20 times.
Element 1 occurs 20 times.


In [7]:
test = data_augment[0]
test = torch.tensor(test)
test = test.to(torch.float32)
test = test.unsqueeze(0)

input = test
input = input.permute(0, 3, 1, 2)

test2 = data_augment[1]
test2 = torch.tensor(test2)
test2 = test2.to(torch.float32)
test2 = test2.unsqueeze(0)

input2 = test2
input2 = input2.permute(0, 3, 1, 2)

print(f"input shape: {input.shape}")
print(f"input2 shape: {input2.shape}")

# Pass the input through the model
model.eval()
p1, p2, z1, z2  = model(input, input2)

print(p1)
print(p2)
print(z1)
print(z2)

input shape: torch.Size([1, 224, 9, 9])
input2 shape: torch.Size([1, 224, 9, 9])
tensor([[ 0.0326,  0.0380, -0.0349,  ...,  0.0245,  0.0202, -0.0495]],
       grad_fn=<AddmmBackward0>)
tensor([[ 0.0326,  0.0381, -0.0348,  ...,  0.0244,  0.0202, -0.0495]],
       grad_fn=<AddmmBackward0>)
tensor([[0.0000, 0.0000, 0.0461,  ..., 0.0217, 0.0108, 0.0000]])
tensor([[0.0000, 0.0000, 0.0462,  ..., 0.0215, 0.0106, 0.0000]])


In [8]:
criterion = nn.CosineSimilarity(dim=1).cuda(gpu)
print(gpu)
optim_params = model.parameters()

momentum = 0.9
weight_decay = 1e-4

optimizer = torch.optim.SGD(optim_params, init_lr,
                                momentum=momentum,
                                weight_decay=weight_decay)

cudnn.benchmark = True
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

augmentation = [
    transforms.RandomHorizontalFlip(),  # Flip along width
    transforms.RandomVerticalFlip(),    # Flip along height
    transforms.RandomRotation(20),      # Rotate image slightly
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize hyperspectral data
]

transform = simsiam.loader.TwoCropsTransform(transforms.Compose(augmentation))

print(data_augment.shape)

0
(40, 9, 9, 224)


In [9]:


class CustomDataset(Dataset):
    def __init__(self, images, transform=None):
        """
        Args:
            images (Tensor or list of Tensors): Preloaded images of shape (N, 9, 9, 224)
            transform (callable, optional): Optional transform to be applied on an image.
        """
        self.images = images  # Assuming it's a list or tensor
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        
        if self.transform:
            img1 = self.transform(img)  # First augmentation
            img2 = self.transform(img)  # Second augmentation
        
            return img1, img2  # Return both augmented versions
        
        return img, img  # If no transform is provided, return the original image twice


# Example usage
pretrain_preloaded_image = data_augment 

pretrain_X_train = torch.tensor(pretrain_preloaded_image)
pretrain_X_train = pretrain_X_train.to(torch.float32)
pretrain_X_train = pretrain_X_train.permute(0, 3, 1, 2)
print(f"X_train shape: {pretrain_X_train.shape}")

# Define transformations if needed
transform = transforms.Compose([
    transforms.Normalize(mean=[0.5], std=[0.5]),  # Example normalization
])

pretrain_train_dataset = CustomDataset(pretrain_X_train, transform=transform)

train_sampler = None

if seeded_run:
    g = torch.Generator()
    g.manual_seed(42)
    
    pretrain_train_loader = DataLoader(
        pretrain_train_dataset,
        batch_size=batch_size,
        shuffle=False,  # set to True if needed
        num_workers=0,
        pin_memory=True,
        drop_last=False,
        generator=g
    )
    print("generate data loader using seed")
else:
    pretrain_train_loader = DataLoader(
        pretrain_train_dataset,
        batch_size=batch_size,
        shuffle=(train_sampler is None),
        num_workers=0,
        pin_memory=True,
        sampler=train_sampler,
        drop_last=False
    )




# 7. Check Output

batch1, batch2 = next(iter(pretrain_train_loader))

print(f"bacth size: {batch1.size()}")
print(f"length batch: {len(batch1)}")  # Should print 2 (Two transformed views per image)
print(f"Train loader size: {len(pretrain_train_loader)}")


X_train shape: torch.Size([40, 224, 9, 9])
bacth size: torch.Size([20, 224, 9, 9])
length batch: 20
Train loader size: 2


In [10]:
def pretrain_adjust_learning_rate(optimizer, init_lr, epoch, epochs):
    """Decay the learning rate based on schedule"""
    cur_lr = init_lr * 0.5 * (1. + math.cos(math.pi * epoch / epochs))
    for param_group in optimizer.param_groups:
        if 'fix_lr' in param_group and param_group['fix_lr']:
            param_group['lr'] = init_lr
        else:
            param_group['lr'] = cur_lr

class Pretrain_AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)
    

class Pretrain_ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'
    
def pretrain_save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [11]:
def pretrain_train(train_loader, model, criterion, optimizer, epoch, device):
    batch_time = Pretrain_AverageMeter('Time', ':6.3f')
    data_time = Pretrain_AverageMeter('Data', ':6.3f')
    losses = Pretrain_AverageMeter('Loss', ':.4f')
    progress = Pretrain_ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses],
        prefix="Epoch: [{}]".format(epoch))

    # switch to train mode
    model.train()
    end = time.time()

    for i, (images1, images2) in enumerate(train_loader):
        data_time.update(time.time() - end)

        input1 = images1.to(device, non_blocking=True)
        input2 = images2.to(device, non_blocking=True)

        p1, p2, z1, z2 = model(x1=input1, x2=input2) 
        loss = -(criterion(p1, z2).mean() + criterion(p2, z1).mean()) * 0.5

        losses.update(loss.item(), input1.size(0))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        batch_time.update(time.time() - end)
        end = time.time()

        if i % 10 == 0:
            progress.display(i)

    # Return average training loss for early stopping
    return losses.avg


In [12]:

# Early stopping parameters
best_loss = float('inf')
patience = 50  # Number of epochs to wait for improvement
patience_counter = 0

start_epoch = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

filename = f"{timestamp}_model.pth.tar"
filepath = f"models/pretrain/{filename}"

for epoch in range(start_epoch, epochs):
    pretrain_adjust_learning_rate(optimizer, init_lr, epoch, epochs)

    # Train and get average loss
    avg_loss = pretrain_train(pretrain_train_loader, model, criterion, optimizer, epoch, device)
    print(f"Epoch {epoch + 1}: Average Training Loss: {avg_loss:.6f}")

    # Early stopping check
    if avg_loss < best_loss:
        best_loss = avg_loss
        patience_counter = 0

        torch.save({
            'epoch': epoch + 1,
            'arch': 'vgg16',
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'best_loss': best_loss
        }, filepath)

        print(f"✅ New best model saved with loss {best_loss:.6f}")

    else:
        patience_counter += 1
        print(f"❌ No improvement. Patience: {patience_counter}/{patience}")

        if patience_counter >= patience:
            print("⏹️ Early stopping triggered.")
            break


Epoch: [0][0/2]	Time  0.620 ( 0.620)	Data  0.002 ( 0.002)	Loss 0.0011 (0.0011)
Epoch 1: Average Training Loss: 0.000845
✅ New best model saved with loss 0.000845
Epoch: [1][0/2]	Time  0.022 ( 0.022)	Data  0.008 ( 0.008)	Loss 0.0001 (0.0001)
Epoch 2: Average Training Loss: 0.000808
✅ New best model saved with loss 0.000808
Epoch: [2][0/2]	Time  0.016 ( 0.016)	Data  0.000 ( 0.000)	Loss -0.0030 (-0.0030)
Epoch 3: Average Training Loss: 0.000400
✅ New best model saved with loss 0.000400
Epoch: [3][0/2]	Time  0.026 ( 0.026)	Data  0.000 ( 0.000)	Loss 0.0043 (0.0043)
Epoch 4: Average Training Loss: 0.000658
❌ No improvement. Patience: 1/50
Epoch: [4][0/2]	Time  0.037 ( 0.037)	Data  0.007 ( 0.007)	Loss -0.0065 (-0.0065)
Epoch 5: Average Training Loss: -0.004973
✅ New best model saved with loss -0.004973
Epoch: [5][0/2]	Time  0.016 ( 0.016)	Data  0.009 ( 0.009)	Loss 0.0026 (0.0026)
Epoch 6: Average Training Loss: 0.002125
❌ No improvement. Patience: 1/50
Epoch: [6][0/2]	Time  0.055 ( 0.055)	Dat

In [13]:
pretrained = rf'C:\Users\Asus TUF\Documents\code\TA\simsiam\simsiam\models\pretrain\{filename}'
print(pretrained)

C:\Users\Asus TUF\Documents\code\TA\simsiam\simsiam\models\pretrain\20250608_094734_model.pth.tar


In [14]:


class VGG16_HSI(nn.Module):
    def __init__(self, num_classes=2):
        super(VGG16_HSI, self).__init__()

         # Custom Convolutional Layer: Process 9x9x224 input
        self.pre_conv = nn.Sequential(
            nn.Conv2d(in_channels=224, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.AdaptiveAvgPool2d((1, 1))  # Reduce to (256, 1, 1)
        )

        # Fully Connected Layer to reshape to (64, 56, 56)
        self.fc = nn.Linear(256 * 1 * 1, 64 * 56 * 56)

        # Load VGG-16 Model
        self.encoder = vgg16(pretrained=False)

        # Remove first VGG-16 conv layer
        self.encoder.features = nn.Sequential(*list(self.encoder.features.children())[1:])

        # Modify classifier to output 2 classes
        self.encoder.classifier[6] = nn.Linear(4096, 2)

    def forward(self, x):
        # print(f'before {x.shape}')
        x = self.pre_conv(x)  # Process hyperspectral input
        x = x.view(x.size(0), -1)  # Flatten

        # print(f'after preconv {x.shape}')
        x = self.fc(x)  # Fully connected layer
        # print(f'after fc {x.shape}')
        # Reshape to (batch_size, 64, 56, 56) before passing to VGG
        x = x.view(x.size(0), 64, 56, 56)
        # print(f'after reshape, before vgg second layer {x.shape}')

        x = self.encoder.features(x)  # Pass to VGG-16
        x = self.encoder.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.encoder.classifier(x)  # Final classification layer

        return x

In [15]:
gpu = 0

print("Use GPU: {} for training".format(gpu))

print("=> creating model")

model_finetune = VGG16_HSI()

# Freeze all layers except the last fully connected layer
for param in model_finetune.pre_conv.parameters():
    param.requires_grad = False  # Freeze convolutional layers
for param in model_finetune.fc.parameters():
    param.requires_grad = False  # Freeze convolutional layers
for param in model_finetune.encoder.features.parameters():
    param.requires_grad = False  # Freeze convolutional layers

for param in model_finetune.encoder.classifier[:-1].parameters():
    param.requires_grad = False  # Freeze all but the last FC layer

# Initialize the last FC layer
# Initialize the last FC layer
torch.nn.init.normal_(model_finetune.encoder.classifier[6].weight, mean=0.0, std=0.01)
torch.nn.init.zeros_(model_finetune.encoder.classifier[6].bias)

# Check which layers are trainable
for name, param in model_finetune.named_parameters():
    print(f"{name}: requires_grad={param.requires_grad}")

Use GPU: 0 for training
=> creating model


d:\FathanAbi\fathanvenv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


pre_conv.0.weight: requires_grad=False
pre_conv.0.bias: requires_grad=False
pre_conv.2.weight: requires_grad=False
pre_conv.2.bias: requires_grad=False
pre_conv.3.weight: requires_grad=False
pre_conv.3.bias: requires_grad=False
pre_conv.5.weight: requires_grad=False
pre_conv.5.bias: requires_grad=False
fc.weight: requires_grad=False
fc.bias: requires_grad=False
encoder.features.1.weight: requires_grad=False
encoder.features.1.bias: requires_grad=False
encoder.features.4.weight: requires_grad=False
encoder.features.4.bias: requires_grad=False
encoder.features.6.weight: requires_grad=False
encoder.features.6.bias: requires_grad=False
encoder.features.9.weight: requires_grad=False
encoder.features.9.bias: requires_grad=False
encoder.features.11.weight: requires_grad=False
encoder.features.11.bias: requires_grad=False
encoder.features.13.weight: requires_grad=False
encoder.features.13.bias: requires_grad=False
encoder.features.16.weight: requires_grad=False
encoder.features.16.bias: requir

In [16]:
if pretrained:
    if os.path.isfile(pretrained):
        print("=> loading checkpoint '{}'".format(pretrained))
        checkpoint = torch.load(pretrained, map_location="cpu")

        # rename moco pre-trained keys
        state_dict = checkpoint['state_dict']

        print("Model state_dict keys:", model.state_dict().keys())  # Debugging
        print("Checkpoint state_dict keys:", state_dict.keys())  
        # for k in list(state_dict.keys()):
        #     print(f"Processing key: {k}")  # Debugging
        #     if k.startswith('module.encoder') and not k.startswith('module.encoder.fc'):
        #         state_dict[k[len("module.encoder."):]] = state_dict[k]
        #     del state_dict[k]

        # Remove the final classification layer from state_dict
        state_dict = {k: v for k, v in state_dict.items() if not k.startswith("encoder.classifier.6")}

        # Load the modified state_dict (ignoring the missing classification layer)
        msg = model.load_state_dict(state_dict, strict=False)

        # Check missing keys
        print("Missing keys:", msg.missing_keys)

        start_epoch = 0
        msg = model.load_state_dict(state_dict, strict=False)
  
        assert set(msg.missing_keys) == {"encoder.classifier.6.weight", "encoder.classifier.6.bias"}

        print("=> loaded pre-trained model '{}'".format(pretrained))
    else:
        print("=> no checkpoint found at '{}'".format(pretrained))



=> no checkpoint found at 'C:\Users\Asus TUF\Documents\code\TA\simsiam\simsiam\models\pretrain\20250608_094734_model.pth.tar'


In [17]:
print(model)

SimSiam(
  (pre_conv): Sequential(
    (0): Conv2d(224, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): AdaptiveAvgPool2d(output_size=(1, 1))
  )
  (fc): Linear(in_features=256, out_features=200704, bias=True)
  (encoder): VGG(
    (features): Sequential(
      (0): ReLU(inplace=True)
      (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): ReLU(inplace=True)
      (6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True

In [18]:
test = data_augment[0]
test = torch.tensor(test)
test = test.to(torch.float32)
test = test.unsqueeze(0)

input = test
input = input.permute(0, 3, 1, 2)

test2 = data_augment[1]
test2 = torch.tensor(test2)
test2 = test2.to(torch.float32)
test2 = test2.unsqueeze(0)

input2 = test2
input2 = input2.permute(0, 3, 1, 2)

print(f"input shape: {input.shape}")
print(f"input2 shape: {input2.shape}")

# Pass the input through the model
model_finetune.eval()
output = model_finetune(input)

print(output)

input shape: torch.Size([1, 224, 9, 9])
input2 shape: torch.Size([1, 224, 9, 9])
tensor([[-4.4198, -1.2717]], grad_fn=<AddmmBackward0>)


In [19]:
lr = 0.1

momentum = 0.9
weight_decay = 0

init_lr = lr * batch_size / 256

torch.cuda.set_device(gpu)
model_finetune = model_finetune.cuda(gpu)

# define loss function (criterion) and optimizer
criterion = nn.CrossEntropyLoss().cuda(gpu)

# optimize only the linear classifier
parameters = list(filter(lambda p: p.requires_grad, model_finetune.parameters()))
assert len(parameters) == 2  # fc.weight, fc.bias

optimizer = torch.optim.SGD(parameters, init_lr,
                            momentum=momentum,
                            weight_decay=weight_decay)

cudnn.benchmark = True

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

finetune_augmentation = [
    transforms.RandomHorizontalFlip(),  # Flip along width
    transforms.RandomVerticalFlip(),    # Flip along height
    transforms.RandomRotation(20),      # Rotate image slightly
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize hyperspectral data
]

finetune_transform = transforms.Compose(finetune_augmentation)

print(data_augment.shape)

(40, 9, 9, 224)


In [20]:

# Example usage
class CustomDatasetFinetune(Dataset):
    def __init__(self, images, labels, transform=None):
        """
        Args:
            images (Tensor or list of Tensors): Preloaded images of shape (N, 9, 9, 224)
            transform (callable, optional): Optional transform to be applied on an image.
        """
        self.images = images  # Assuming it's a list or tensor
        self.transform = transform
        self.label = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        label = self.label[idx]
        
        if self.transform:
            img1 = self.transform(img)  # First augmentation
        
            return img1, label  # Return both augmented versions
        
        return img, label  # If no transform is provided, return the original image twice
    
finetune_preloaded_images = data_augment  
finetune_X = torch.tensor(finetune_preloaded_images)
finetune_X= finetune_X.to(torch.float32)
finetune_X = finetune_X.permute(0, 3, 1, 2)
print(f"finetune_X_train shape: {finetune_X.shape}")

finetune_y = torch.tensor(label_augment)
#
# Define transformations if needed
finetune_transform = transforms.Compose(finetune_augmentation)
testSize = test_size
finetune_X_train, finetune_X_val, finetune_y_train, finetune_y_val = train_test_split(finetune_X, finetune_y, test_size = testSize, random_state=42, stratify=finetune_y)
print(f"Train shape: {finetune_X_train.shape}, Validation shape: {finetune_X_val.shape}")

finetune_train_dataset = CustomDatasetFinetune(finetune_X_train, finetune_y_train, transform=finetune_transform)
finetune_val_dataset = CustomDatasetFinetune(finetune_X_val, finetune_y_val, transform=finetune_transform)

train_sampler = None

if seeded_run:
    g = torch.Generator()
    g.manual_seed(42)
    
    finetune_train_loader = DataLoader(
        finetune_train_dataset,
        batch_size=batch_size,
        shuffle=False,  # set to True if needed
        num_workers=0,
        pin_memory=True,
        drop_last=False,
        generator=g
    )
    finetune_val_loader = DataLoader(
        finetune_val_dataset,
        batch_size=batch_size,
        shuffle=False,  # set to True if needed
        num_workers=0,
        pin_memory=True,
        drop_last=False,
        generator=g
    )
    
    print("generate data loader using seed")
else:
    finetune_train_loader = DataLoader(finetune_train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True, drop_last=False)
    finetune_val_loader = DataLoader(finetune_val_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True, drop_last=False)



# 7. Check Output

batch1 = next(iter(finetune_train_loader))

print(batch1[1].size())
print(f"Train loader size: {len(finetune_train_loader)}, Validation loader size: {len(finetune_val_loader)}")


finetune_X_train shape: torch.Size([40, 224, 9, 9])
Train shape: torch.Size([20, 224, 9, 9]), Validation shape: torch.Size([20, 224, 9, 9])
torch.Size([20])
Train loader size: 1, Validation loader size: 1


In [21]:
def finetune_train(train_loader, model, criterion, optimizer, epoch):
    batch_time = FinetuneAverageMeter('Time', ':6.3f')
    data_time = FinetuneAverageMeter('Data', ':6.3f')
    losses = FinetuneAverageMeter('Loss', ':.4e')
    top1 = FinetuneAverageMeter('Acc@1', ':6.2f')
    progress = FinetuneProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1],
        prefix="Epoch: [{}]".format(epoch))

    """
    Switch to eval mode:
    Under the protocol of linear classification on frozen features/models,
    it is not legitimate to change any part of the pre-trained model.
    BatchNorm in train mode may revise running mean/std (even if it receives
    no gradient), which are part of the model parameters too.
    """
    model.eval()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        gpu = 0
        images = images.cuda(gpu, non_blocking=True)
        target = target.cuda(gpu, non_blocking=True)

        # compute output
        output = model(images)
        loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, = finetune_accuracy(output, target, topk=(1,))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))


        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        print_freq = 10
        if i % print_freq == 0:
            progress.display(i)


def finetune_validate(val_loader, model, criterion):
    batch_time = FinetuneAverageMeter('Time', ':6.3f')
    losses = FinetuneAverageMeter('Loss', ':.4e')
    top1 = FinetuneAverageMeter('Acc@1', ':6.2f')
  
    progress = FinetuneProgressMeter(
        len(val_loader),
        [batch_time, losses, top1],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
       
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
      
            gpu = 0
            images = images.cuda(gpu, non_blocking=True)
            target = target.cuda(gpu, non_blocking=True)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, = finetune_accuracy(output, target, topk=(1,))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            # top5.update(acc5[0], images.size(0))
            print(f"in validation finction {acc1}")
            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            print_freq = 10
            if i % print_freq == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f}'.format(top1=top1))


    return top1.avg


def finetune_save_checkpoint(timestamp, epoch, state, is_best, filename='models/checkpoint.pth.tar'):
    filename='models/finetune/{}_model.pth.tar'.format(timestamp)
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')


def finetune_sanity_check(state_dict, pretrained_weights):
    """
    Linear classifier should not change any weights other than the linear layer.
    This sanity check asserts nothing wrong happens (e.g., BN stats updated).
    """
    print("=> loading '{}' for sanity check".format(pretrained_weights))
    checkpoint = torch.load(pretrained_weights, map_location="cpu")
    state_dict_pre = checkpoint['state_dict']

    for k in list(state_dict.keys()):
        # Ignore fc layer
        if 'fc.weight' in k or 'fc.bias' in k:
            continue

        # Adjust key mapping to match checkpoint format
        k_pre = k.replace('module.encoder.', '')  # Remove unnecessary prefix

        # Skip missing keys
        if k_pre not in state_dict_pre:
            print(f"Warning: {k_pre} not found in pretrained model. Skipping...")
            continue

        # Check if tensor shapes match before comparing values
        if state_dict[k].shape != state_dict_pre[k_pre].shape:
            print(f"Warning: Shape mismatch for {k}: {state_dict[k].shape} vs {state_dict_pre[k_pre].shape}. Skipping...")
            continue

        # Assert that the weights remain unchanged
        assert ((state_dict[k].cpu() == state_dict_pre[k_pre]).all()), \
            '{} is changed in linear classifier training.'.format(k)

    print("=> sanity check passed.")



class FinetuneAverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class FinetuneProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'


def finetune_adjust_learning_rate(optimizer, init_lr, epoch, epochs):
    """Decay the learning rate based on schedule"""
    cur_lr = init_lr * 0.5 * (1. + math.cos(math.pi * epoch / epochs))
    for param_group in optimizer.param_groups:
        param_group['lr'] = cur_lr


def finetune_accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res


In [22]:
for name, param in model_finetune.named_parameters():
    print(f"{name}: requires_grad={param.requires_grad}")

pre_conv.0.weight: requires_grad=False
pre_conv.0.bias: requires_grad=False
pre_conv.2.weight: requires_grad=False
pre_conv.2.bias: requires_grad=False
pre_conv.3.weight: requires_grad=False
pre_conv.3.bias: requires_grad=False
pre_conv.5.weight: requires_grad=False
pre_conv.5.bias: requires_grad=False
fc.weight: requires_grad=False
fc.bias: requires_grad=False
encoder.features.1.weight: requires_grad=False
encoder.features.1.bias: requires_grad=False
encoder.features.4.weight: requires_grad=False
encoder.features.4.bias: requires_grad=False
encoder.features.6.weight: requires_grad=False
encoder.features.6.bias: requires_grad=False
encoder.features.9.weight: requires_grad=False
encoder.features.9.bias: requires_grad=False
encoder.features.11.weight: requires_grad=False
encoder.features.11.bias: requires_grad=False
encoder.features.13.weight: requires_grad=False
encoder.features.13.bias: requires_grad=False
encoder.features.16.weight: requires_grad=False
encoder.features.16.bias: requir

In [23]:
best_acc1 = 0.0
patience = 50  # Adjust as needed
patience_counter = 0
# timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
start_epoch = 0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_finetune.to(device)


for epoch in range(start_epoch, epochs):
    finetune_adjust_learning_rate(optimizer, init_lr, epoch, epochs)

    # Train for one epoch
    finetune_train(finetune_train_loader, model_finetune, criterion, optimizer, epoch)

    # Evaluate on validation set
    acc1 = finetune_validate(finetune_val_loader, model_finetune, criterion)

    # Check if current accuracy is the best
    is_best = acc1 > best_acc1

    if is_best:
        best_acc1 = acc1
        patience_counter = 0

        # Save best model only
        finetune_save_checkpoint(timestamp, epoch, {
            'epoch': epoch + 1,
            'arch': 'vgg16',
            'state_dict': model_finetune.state_dict(),
            'best_acc1': best_acc1,
            'optimizer': optimizer.state_dict(),
        }, is_best=True)

        print(f"✅ Epoch {epoch+1}: New best Acc@1: {best_acc1:.2f}. Model saved.")

    else:
        patience_counter += 1
        print(f"❌ Epoch {epoch+1}: No improvement. Patience counter: {patience_counter}/{patience}")

        if patience_counter >= patience:
            print(f"⏹️ Early stopping triggered at epoch {epoch+1}. Best Acc@1: {best_acc1:.2f}")
            break


Epoch: [0][0/1]	Time  0.026 ( 0.026)	Data  0.008 ( 0.008)	Loss 4.8438e+00 (4.8438e+00)	Acc@1  50.00 ( 50.00)
in validation finction tensor([50.], device='cuda:0')
Test: [0/1]	Time  0.057 ( 0.057)	Loss 8.9707e+02 (8.9707e+02)	Acc@1  50.00 ( 50.00)
 * Acc@1 50.000
✅ Epoch 1: New best Acc@1: 50.00. Model saved.
Epoch: [1][0/1]	Time  0.026 ( 0.026)	Data  0.022 ( 0.022)	Loss 1.1778e+03 (1.1778e+03)	Acc@1  50.00 ( 50.00)
in validation finction tensor([50.], device='cuda:0')
Test: [0/1]	Time  0.011 ( 0.011)	Loss 5.9865e+02 (5.9865e+02)	Acc@1  50.00 ( 50.00)
 * Acc@1 50.000
❌ Epoch 2: No improvement. Patience counter: 1/50
Epoch: [2][0/1]	Time  0.019 ( 0.019)	Data  0.013 ( 0.013)	Loss 7.8765e+02 (7.8765e+02)	Acc@1  50.00 ( 50.00)
in validation finction tensor([50.], device='cuda:0')
Test: [0/1]	Time  0.007 ( 0.007)	Loss 8.5836e+02 (8.5836e+02)	Acc@1  50.00 ( 50.00)
 * Acc@1 50.000
❌ Epoch 3: No improvement. Patience counter: 2/50
Epoch: [3][0/1]	Time  0.016 ( 0.016)	Data  0.012 ( 0.012)	Loss 8

In [24]:
end_time = time.time()
print(f"Run time: {end_time - start_time:.4f} seconds")

print(timestamp)

Run time: 68.9179 seconds
20250608_094734
